In [26]:
import sys  
sys.path.insert(0, '..')

from datetime import date, timedelta

from config import Config

import pandas as pd
from sqlalchemy import create_engine

db = create_engine(Config.SQLALCHEMY_DATABASE_URI)

In [103]:
registrace = pd.read_sql_query(
    """
    select *
    from ockovani_registrace
    where import_id = 17
    """,
    db
)
# registrace

In [99]:
registrace_ext = registrace
registrace_ext['datum_rezervace_fix'] = registrace['datum_rezervace'].where(registrace['datum_rezervace'] != date(1970, 1, 1))
registrace_ext['fronta'] = registrace[['pocet']].where(registrace['rezervace'] == False)
registrace_ext['registrace_7'] = registrace[['pocet']].where(registrace['datum'] >= date.today() - timedelta(7))
registrace_ext['registrace_7_rez'] = registrace[['pocet']].where((registrace['rezervace'] == True) & (registrace['datum'] >= date.today() - timedelta(7)))
registrace_ext['registrace_14'] = registrace[['pocet']].where(registrace['datum'] >= date.today() - timedelta(14))
registrace_ext['registrace_14_rez'] = registrace[['pocet']].where((registrace['rezervace'] == True) & (registrace['datum'] >= date.today() - timedelta(14)))
registrace_ext['registrace_30'] = registrace[['pocet']].where(registrace['datum'] >= date.today() - timedelta(30))
registrace_ext['registrace_30_rez'] = registrace[['pocet']].where((registrace['rezervace'] == True) & (registrace['datum'] >= date.today() - timedelta(30)))
registrace_ext['cekani'] = (registrace['datum_rezervace_fix'] - registrace['datum']).dt.days
registrace_ext['rezervace_7'] = registrace[['pocet']].where((registrace['rezervace'] == True) & (registrace['datum_rezervace_fix'] >= date.today() - timedelta(7))) 
registrace_ext['rezervace_7_x_cekani'] = registrace['cekani'] * registrace['rezervace_7']


In [100]:
registrace_grp = registrace_ext.groupby(['vekova_skupina', 'povolani']).sum()
# registrace_grp

In [101]:
registrace_met = registrace_grp
registrace_met['uspestost_7'] = registrace_grp['registrace_7_rez'] / registrace_grp['registrace_7']
registrace_met['uspestost_14'] = registrace_grp['registrace_14_rez'] / registrace_grp['registrace_14']
registrace_met['uspestost_30'] = registrace_grp['registrace_30_rez'] / registrace_grp['registrace_30']
registrace_met['prumer_cekani'] = registrace_grp['rezervace_7_x_cekani'] / registrace_grp['rezervace_7']

registrace_met.reset_index()

,vekova_skupina,povolani,rezervace,pocet,import_id,fronta,registrace_7,registrace_7_rez,registrace_14,registrace_14_rez,registrace_30,registrace_30_rez,cekani,rezervace_7,rezervace_7_x_cekani,uspestost_7,uspestost_14,uspestost_30,prumer_cekani
0,0-17,Pedagogický pracovník/nepedagogický zaměstnane...,4,79,1207,75.0,6.0,0.0,21.0,0.0,79.0,4.0,7.0,0.0,0.0,0.000000,0.000000,0.050633,NaN
1,0-17,Zdravotnický pracovník dle §76 a §77 zákona 37...,4,6,102,2.0,1.0,0.0,3.0,2.0,5.0,3.0,7.0,1.0,2.0,0.000000,0.666667,0.600000,2.000000
2,18-24,Pedagogický pracovník/nepedagogický zaměstnane...,395,3350,21964,2500.0,310.0,48.0,1092.0,210.0,3350.0,850.0,1369.0,151.0,967.0,0.154839,0.192308,0.253731,6.403974
3,18-24,Zdravotnický pracovník dle §76 a §77 zákona 37...,683,1220,15385,295.0,193.0,79.0,483.0,280.0,940.0,680.0,5353.0,172.0,833.0,0.409326,0.579710,0.723404,4.843023
4,18-24,neuvedeno,2,7,119,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
5,25-29,Pedagogický pracovník/nepedagogický zaměstnane...,661,6912,33218,5092.0,668.0,92.0,2194.0,485.0,6912.0,1820.0,2447.0,335.0,2149.0,0.137725,0.221057,0.263310,6.414925
6,25-29,Zdravotnický pracovník dle §76 a §77 zákona 37...,1388,2815,31008,559.0,258.0,133.0,768.0,484.0,1868.0,1457.0,13193.0,305.0,2614.0,0.515504,0.630208,0.779979,8.570492
7,25-29,neuvedeno,2,7,119,5.0,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,NaN,NaN,NaN,NaN
8,30-34,Pedagogický pracovník/nepedagogický zaměstnane...,772,8325,36720,5927.0,886.0,143.0,2664.0,649.0,8325.0,2398.0,2985.0,542.0,3586.0,0.161400,0.243619,0.288048,6.616236
9,30-34,Zdravotnický pracovník dle §76 a §77 zákona 37...,1522,3127,34425,657.0,269.0,105.0,845.0,496.0,2088.0,1589.0,15254.0,341.0,3455.0,0.390335,0.586982,0.761015,10.131965
